<a href="https://colab.research.google.com/github/djfLtC0dr/python-playground/blob/main/DASC522/djfDASC522_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

a. Break your code into logical chunks => SOP imports

In [9]:
# importing libraries
import statsmodels.api as sm
import pandas as pd 

# loading the training dataset 
data = pd.read_csv('/home/LRS_Pre_Assessment_trimmed_rank.csv', index_col = 0)
print(data)

              Rank  Age Gender  ORS_total  PTSD_score  Sleep_score  \
Flight                                                               
D       JrEnlisted   40      M        9.8           5            8   
D       JrEnlisted   22      M        5.0          22           10   
D       JrEnlisted   20      M        9.6           1            3   
M       JrEnlisted   21      F        5.0          12           13   
D       JrEnlisted   27      M        8.3           0            6   
...            ...  ...    ...        ...         ...          ...   
M       SrEnlisted   39      F        3.3          28           12   
M       JrEnlisted   21      F        4.7          13            7   
M       JrEnlisted   23      F        8.7           0            6   
M       JrEnlisted   20      M        5.7          12            4   
CC      SrEnlisted   34      M        9.0           8            3   

        survey_Burnout  survey_FitSat   BMI  BodyFatPerc  MusclePerc  \
Flight           

b. Drop the “flight” column and one-hot-encode the “rank” & “gender” *columns*

In [ ]:
data.drop('Flight', axis=1, inplace=True)
data_encoded = pd.get_dummies(data, columns = ['Rank','Gender'])

c. Data Understanding